<h1>Diagnosis Data</h1>

Data is from CDC AtlasPlus website.

In [2]:
#Dependencies
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

#Config variables
from config import remote_db_endpoint, remote_db_port
from config import remote_hiv_dbname, remote_hiv_dbuser, remote_hiv_dbpwd
# from config import local_hiv_dbuser, local_hiv_dbpwd, local_db_endpoint, local_db_port, local_hiv_dbname

Create engine to pass in Cloud MySQL Database Connection on AWS

In [ ]:
#Cloud MySQL Database Connection on AWS
engine = create_engine(f'mysql://{remote_hiv_dbuser}:{remote_hiv_dbpwd}@{remote_db_endpoint}:{remote_db_port}/{remote_hiv_dbname}')

In [ ]:
#Create a remote database engine connection
conn = engine.connect()

<h3>Overall Diagnosis</h3>

In [3]:
diag_all_files = glob.glob("Resources/Diag_All*.csv")
diag_all_files

['Resources/Diag_All_County.csv',
 'Resources/Diag_All_National.csv',
 'Resources/Diag_All_State.csv']

In [4]:
#Check data is reading in and how it looks
diag_test = pd.read_csv('Resources/Diag_All_National.csv')
# diag_test = pd.read_csv('Resources/Diag_All_State')
# diag_test = pd.read_csv('Resources/Diag_All_County.csv')

diag_test.head()
# diag_test.columns

,Year,Geography,FIPS,Cases,Rate per 100000,Population,Indicator
0,2018,United States,NaN,"37,286",13.6,"274,605,948",HIV diagnoses
1,2017,United States,NaN,"38,226",14.0,"272,460,904",HIV diagnoses
2,2016,United States,NaN,"39,520",14.6,"270,301,427",HIV diagnoses
3,2015,United States,NaN,"39,796",14.8,"268,005,522",HIV diagnoses
4,2014,United States,NaN,"39,963",15.0,"265,732,678",HIV diagnoses


In [5]:
def parsed(x):
    try:
        return int(x)
    except:
        return x
i=0

for f in diag_all_files:
    diag_all_df = pd.read_csv(f)
    #Shorten and rename columns
    diag_all_df = diag_all_df.rename(columns={'Year':'Year','Geography':'Full Name', 'Cases':'All Diag Cases','Rate per 100000':'All Diag Rates'
    })

    diag_all_df = diag_all_df.replace({'Data suppressed': np.nan, ',': ''}, regex=True)
    diag_all_df = diag_all_df.apply(lambda x: parsed(x))
    diag_all_df['Year'] = diag_all_df['Year'].apply(lambda x: parsed(x))
    if i>1:
        diag_all_df1 = diag_all_df
    elif i>0:
        diag_all_df0 = diag_all_df
    else:
        diag_all_df2 = diag_all_df
    i = i + 1

# Concat all the df's together into one
diag_all_data = pd.concat([diag_all_df0,diag_all_df1,diag_all_df2]).reset_index(drop=True)
diag_all_data.head()

,Year,Full Name,FIPS,All Diag Cases,All Diag Rates,Population,Indicator
0,2018,United States,NaN,37286,13.6,274605948,HIV diagnoses
1,2017,United States,NaN,38226,14,272460904,HIV diagnoses
2,2016,United States,NaN,39520,14.6,270301427,HIV diagnoses
3,2015,United States,NaN,39796,14.8,268005522,HIV diagnoses
4,2014,United States,NaN,39963,15,265732678,HIV diagnoses


In [6]:
diag_all_data.to_csv('Output/diag_all.csv',index=False)

<h3>Diagnosis Data by Age</h3>

In [7]:
diag_age_files = sorted(glob.glob("Resources/Diag_Age*.csv"))
diag_age_files

['Resources/Diag_Age_County_CandR.csv',
 'Resources/Diag_Age_National_CandR.csv',
 'Resources/Diag_Age_State_CandR.csv']

In [8]:
#Check data is reading in and how it looks
# diag_age_test = pd.read_csv('Resources/Diag_Age_National_CandR.csv')
diag_age_test = pd.read_csv('Resources/Diag_Age_State_CandR.csv')
# diag_age_test = pd.read_csv('Resources/Diag_Age_County_CandR.csv')

diag_age_test.head()
# diag_age_test.columns

,Year,Geography,13-24,Unnamed: 3,25-34,Unnamed: 5,35-44,Unnamed: 7,45-54,Unnamed: 9,55+,Unnamed: 11
0,NaN,NaN,Cases,Rate per 100000,Cases,Rate per 100000,Cases,Rate per 100000,Cases,Rate per 100000,Cases,Rate per 100000
1,2008.0,Alabama,168,21.4,185,30.3,165,26.5,143,21.3,48,4.1
2,2008.0,Alaska,5,3.7,11,10.7,10,10.8,11,10.4,2,1.6
3,2008.0,Arizona,110,10.4,213,22.5,203,23.3,117,13.8,36,2.4
4,2008.0,Arkansas,47,10,69,18,57,15.3,44,11.1,21,2.9


In [9]:
def parsed(x):
    try:
        return int(x)
    except:
        return x
i=0

for f in diag_age_files:
    diag_age_df = pd.read_csv(f)
    #Rename columns 
    diag_age_df = diag_age_df.rename(columns={'Year':'Year','Geography':'Full Name','13-24':'13-24 Diag Cases','Unnamed: 3':'13-24 Diag Rates',
                                              '25-34':'25-34 Diag Cases','Unnamed: 5':'25-34 Diag Rates',
                                              '35-44':'35-44 Diag Cases','Unnamed: 7':'35-44 Diag Rates',
                                              '45-54':'45-54 Diag Cases','Unnamed: 9':'45-54 Diag Rates',
                                              '55+':'55+ Diag Cases','Unnamed: 11':'55+ Diag Rates'
    })
    diag_age_df = diag_age_df.iloc[1:].replace({'Data suppressed': np.nan, ',': ''}, regex=True)
    diag_age_df = diag_age_df.apply(lambda x: parsed(x))
    diag_age_df['Year'] = diag_age_df['Year'].apply(lambda x: parsed(x))
    if i>1:
        diag_age_df1 = diag_age_df
    elif i>0:
        diag_age_df0 = diag_age_df
    else:
        diag_age_df2 = diag_age_df
    i = i + 1

# print(diag_race_df0.head())
# print(diag_race_df1.head())
# print(diag_race_df2.head())
diag_age_data = pd.concat([diag_age_df0,diag_age_df1,diag_age_df2]).reset_index(drop=True)
diag_age_data.head()

,Year,Full Name,13-24 Diag Cases,13-24 Diag Rates,25-34 Diag Cases,25-34 Diag Rates,35-44 Diag Cases,35-44 Diag Rates,45-54 Diag Cases,45-54 Diag Rates,55+ Diag Cases,55+ Diag Rates
0,2008,United States,8577,16.8,12538,30.6,12630,29.9,9385,21.2,4160,5.7
1,2009,United States,8802,17.2,11984,28.8,11392,27.4,8652,19.4,3921,5.3
2,2010,United States,9123,17.6,11703,28.4,10132,24.7,8227,18.3,3882,5
3,2011,United States,9060,17.4,11504,27.5,9126,22.5,7800,17.4,3821,4.8
4,2012,United States,9058,17.3,11816,27.9,8510,21,7396,16.7,3749,4.6


In [10]:
diag_age_data.to_csv('Output/diag_age.csv',index=False)

<h3>Diagnosis Data by Race</h3>

In [11]:
diag_race_files = glob.glob("Resources/Diag_Race*.csv")
diag_race_files

['Resources/Diag_Race_National_CandR.csv',
 'Resources/Diag_Race_County_CandR.csv',
 'Resources/Diag_Race_State_CandR.csv']

In [12]:
#Check data is reading in and how it looks
# diag_race_test = pd.read_csv('Resources/Diag_Race_National_CandR.csv')
# diag_race_test = pd.read_csv('Resources/Diag_Race_State_CandR.csv')
diag_race_test = pd.read_csv('Resources/Diag_Race_County_CandR.csv')

diag_race_test.head()
diag_race_test.columns

Index(['Year', 'Geography', 'American Indian/Alaska Native', 'Unnamed: 3',
       'Asian', 'Unnamed: 5', 'Black/African American', 'Unnamed: 7',
       'Hispanic/Latino', 'Unnamed: 9', 'Multiple races', 'Unnamed: 11',
       'Native Hawaiian/Other Pacific Islander', 'Unnamed: 13', 'White',
       'Unnamed: 15'],
      dtype='object')

In [13]:
def parsed(x):
    try:
        return int(x)
    except:
        return x
i=0

for f in diag_race_files:
    diag_race_df = pd.read_csv(f)
    #Shorten and rename columns (for usability) American Indian/Alaska Native=AI_AN, Asian, Black/African American=Black, Hispanic/Latino=Latino, Multiple races=Multi, Native Hawaiian/Other Pacific Islander=NH_OPI, White
    diag_race_df = diag_race_df.rename(columns={'Year':'Year','Geography':'Full Name','American Indian/Alaska Native':'AI_AN Diag Cases','Unnamed: 3':'AI_AN Diag Rates',
                                                'Asian':'Asian Diag Cases','Unnamed: 5':'Asian Diag Rates',
                                                'Black/African American':'Black Diag Cases','Unnamed: 7':'Black Diag Rates',
                                                'Hispanic/Latino':'Latino Diag Cases','Unnamed: 9':'Latino Diag Rates',
                                                'Multiple races':'Multi Diag Cases','Unnamed: 11':'Multi Diag Rates',
                                                'Native Hawaiian/Other Pacific Islander':'NH_OPI Diag Cases','Unnamed: 13':'NH_OPI Diag Rates',
                                                'White':'White Diag Cases','Unnamed: 15':'White Diag Rates',
                                               })

    diag_race_df = diag_race_df.iloc[1:].replace({'Data suppressed': np.nan, ',': ''}, regex=True)
    diag_race_df = diag_race_df.apply(lambda x: parsed(x))
    diag_race_df['Year'] = diag_race_df['Year'].apply(lambda x: parsed(x))
    if i>1:
        diag_race_df1 = diag_race_df
    elif i>0:
        diag_race_df2 = diag_race_df
    else:
        diag_race_df0 = diag_race_df
    i = i + 1

# print(diag_race_df0.head())
# print(diag_race_df1.head())
# print(diag_race_df2.head())
diag_race_data = pd.concat([diag_race_df0,diag_race_df1,diag_race_df2]).reset_index(drop=True)
diag_race_data.head()

,Year,Full Name,AI_AN Diag Cases,AI_AN Diag Rates,Asian Diag Cases,Asian Diag Rates,Black Diag Cases,Black Diag Rates,Latino Diag Cases,Latino Diag Rates,Multi Diag Cases,Multi Diag Rates,NH_OPI Diag Cases,NH_OPI Diag Rates,White Diag Cases,White Diag Rates
0,2008,United States,147,7.9,659,6,21719,72.5,9820,28.3,2327,82.1,48,13.7,12570,7.4
1,2009,United States,142,7.5,646,5.7,20346,67.2,9523,26.7,2204,75.5,47,13.2,11843,6.9
2,2010,United States,153,8.5,665,5.4,19385,63.1,9286,24.4,2111,59.3,46,11.5,11421,6.7
3,2011,United States,134,7.3,708,5.5,18457,59.3,9091,23.3,1955,53,52,12.7,10914,6.4
4,2012,United States,168,9.1,760,5.7,17696,56.2,9147,22.9,1892,49.5,46,11,10820,6.4


In [14]:
diag_race_data.to_csv('Output/diag_race.csv',index=False)

<h3>Diagnosis Data by Transmission</h3>

In [15]:
diag_tran_files = sorted(glob.glob("Resources/Diag_Tran*.csv"))
diag_tran_files

['Resources/Diag_Tran_County_C.csv',
 'Resources/Diag_Tran_National_C.csv',
 'Resources/Diag_Tran_State_C.csv']

In [16]:
#Check data is reading in and how it looks
diag_tran_test = pd.read_csv('Resources/Diag_Tran_National_C.csv')
# diag_tran_test = pd.read_csv('Resources/Diag_Tran_State_C.csv')
# diag_tran_test = pd.read_csv('Resources/Diag_Tran_County_C.csv')

diag_tran_test.head()
# diag_tran_test.columns

,Year,Geography,Heterosexual contact,Injection drug use,Male-to-male sexual contact,Male-to-male sexual contact and injection drug use,Other
0,NaN,NaN,Cases,Cases,Cases,Cases,Cases
1,2008.0,United States,"14,189","4,455","26,537","2,017",93
2,2009.0,United States,"12,926","3,793","26,070","1,876",85
3,2010.0,United States,"12,032","3,307","25,895","1,752",82
4,2011.0,United States,"11,171","2,827","25,619","1,618",77


In [17]:
def parsed(x):
    try:
        return int(x)
    except:
        return x
i=0

for f in diag_tran_files:
    diag_tran_df = pd.read_csv(f)
    #Shorten and Rename columns Heterosexual contact=M2F   Injection drug use=ID   Male-to-male sexual contact=M2M   Male-to-male sexual contact and injection drug use=M2MID   Other=O
    diag_tran_df = diag_tran_df.rename(columns={'Year':'Year', 'Geography':'Full Name',
                                                'Heterosexual contact':'M2F Diag Cases',
                                                'Injection drug use':'ID Diag Cases',
                                                'Male-to-male sexual contact':'M2M Diag Cases',
                                                'Male-to-male sexual contact and injection drug use':'M2MID Diag Cases',
                                                'Other':'O Diag Cases'
                                               })
    diag_tran_df = diag_tran_df.iloc[1:].replace({'Data suppressed': np.nan, ',': ''}, regex=True)
    diag_tran_df = diag_tran_df.apply(lambda x: parsed(x))
    diag_tran_df['Year'] = diag_tran_df['Year'].apply(lambda x: parsed(x))
    if i>1:
        diag_tran_df1 = diag_tran_df
    elif i>0:
        diag_tran_df0 = diag_tran_df
    else:
        diag_tran_df2 = diag_tran_df
    i = i + 1

# print(diag_tran_df0.head())
# print(diag_tran_df1.head())
# print(diag_tran_df2.head())
diag_tran_data = pd.concat([diag_tran_df0,diag_tran_df1,diag_tran_df2]).reset_index(drop=True)
diag_tran_data.head()

,Year,Full Name,M2F Diag Cases,ID Diag Cases,M2M Diag Cases,M2MID Diag Cases,O Diag Cases
0,2008,United States,14189,4455,26537,2017,93
1,2009,United States,12926,3793,26070,1876,85
2,2010,United States,12032,3307,25895,1752,82
3,2011,United States,11171,2827,25619,1618,77
4,2012,United States,10423,2549,25915,1565,77


In [18]:
diag_tran_data.to_csv('Output/diag_tran.csv',index=False)

<h3>Diagnosis Data by Sex</h3>

In [19]:
diag_sex_files = glob.glob("Resources/Diag_Sex*.csv")
diag_sex_files

['Resources/Diag_Sex_County_CandR.csv',
 'Resources/Diag_Sex_National_CandR.csv',
 'Resources/Diag_Sex_State_CandR.csv']

In [20]:
#Check data is reading in and how it looks
diag_sex_test = pd.read_csv('Resources/Diag_Sex_National_CandR.csv')
# diag_sex_test = pd.read_csv('Resources/Diag_Sex_State_CandR.csv')
# diag_sex_test = pd.read_csv('Resources/Diag_Sex_County_CandR.csv')

diag_sex_test.head()
# diag_sex_test.columns

,Year,Geography,Female,Unnamed: 3,Male,Unnamed: 5
0,NaN,NaN,Cases,Rate per 100000,Cases,Rate per 100000
1,2008.0,United States,"11,397",8.9,"35,893",29.2
2,2009.0,United States,"10,203",7.9,"34,548",27.9
3,2010.0,United States,"9,454",7.2,"33,613",26.9
4,2011.0,United States,"8,717",6.6,"32,594",25.8


In [21]:
def parsed(x):
    try:
        return int(x)
    except:
        return x
i=0

for f in diag_sex_files:
    diag_sex_df = pd.read_csv(f)
    #Shorten and Rename columns Heterosexual contact=M2F   Injection drug use=ID   Male-to-male sexual contact=M2M   Male-to-male sexual contact and injection drug use=M2MID   Other=O
    diag_sex_df = diag_sex_df.rename(columns={'Year':'Year','Geography':'Full Name',
                                              'Female':'F Diag Cases','Unnamed: 3':'F Diag Rates',
                                              'Male':'M Diag Cases','Unnamed: 5':'M Diag Rates',
                                               })
    diag_sex_df = diag_sex_df.iloc[1:].replace({'Data suppressed': np.nan, ',': ''}, regex=True)
    diag_sex_df = diag_sex_df.apply(lambda x: parsed(x))
    diag_sex_df['Year'] = diag_sex_df['Year'].apply(lambda x: parsed(x))
    if i>1:
        diag_sex_df1 = diag_sex_df
    elif i>0:
        diag_sex_df0 = diag_sex_df
    else:
        diag_sex_df2 = diag_sex_df
    i = i + 1

# print(diag_sex_df0.head())
# print(diag_sex_df1.head())
# print(diag_sex_df2.head())
diag_sex_data = pd.concat([diag_sex_df0,diag_sex_df1,diag_sex_df2]).reset_index(drop=True)
diag_sex_data.head()

,Year,Full Name,F Diag Cases,F Diag Rates,M Diag Cases,M Diag Rates
0,2008,United States,11397,8.9,35893,29.2
1,2009,United States,10203,7.9,34548,27.9
2,2010,United States,9454,7.2,33613,26.9
3,2011,United States,8717,6.6,32594,25.8
4,2012,United States,8125,6.1,32404,25.4


In [22]:
diag_sex_data.to_csv('Output/diag_sex.csv',index=False)

<h3>Database Diagnosis</h3>
Merge all Diagnosis Data Files into one 'master_diag.csv' for loading to database

In [23]:
diag_all_data = diag_all_data[['Indicator','Year','Full Name','FIPS','Population','All Diag Cases','All Diag Rates']]
diag_all_data.head()

,Indicator,Year,Full Name,FIPS,Population,All Diag Cases,All Diag Rates
0,HIV diagnoses,2018,United States,NaN,274605948,37286,13.6
1,HIV diagnoses,2017,United States,NaN,272460904,38226,14
2,HIV diagnoses,2016,United States,NaN,270301427,39520,14.6
3,HIV diagnoses,2015,United States,NaN,268005522,39796,14.8
4,HIV diagnoses,2014,United States,NaN,265732678,39963,15


In [24]:
diag_merge = pd.merge(diag_all_data, diag_sex_data, how='outer', on=['Year','Full Name'])
diag_merge1 = pd.merge(diag_merge, diag_age_data, how='outer', on=['Year','Full Name'])
diag_merge2 = pd.merge(diag_merge1, diag_race_data, how='outer', on=['Year','Full Name'])
diag_master_data = pd.merge(diag_merge2, diag_tran_data, how='outer', on=['Year','Full Name'])
diag_master_data.head()

,Indicator,Year,Full Name,FIPS,Population,All Diag Cases,All Diag Rates,F Diag Cases,F Diag Rates,M Diag Cases,...,Multi Diag Rates,NH_OPI Diag Cases,NH_OPI Diag Rates,White Diag Cases,White Diag Rates,M2F Diag Cases,ID Diag Cases,M2M Diag Cases,M2MID Diag Cases,O Diag Cases
0,HIV diagnoses,2018,United States,NaN,274605948,37286,13.6,7139,5.1,30147,...,18.7,69,14.4,9606,5.6,8778,2478,24650,1305,76
1,HIV diagnoses,2017,United States,NaN,272460904,38226,14,7284,5.2,30942,...,24,53,11.3,9808,5.7,9056,2343,25404,1347,76
2,HIV diagnoses,2016,United States,NaN,270301427,39520,14.6,7497,5.4,32023,...,29.8,41,8.9,9993,5.8,9759,2192,26090,1409,71
3,HIV diagnoses,2015,United States,NaN,268005522,39796,14.8,7389,5.4,32407,...,34.1,72,16,10258,6,9584,2328,26387,1432,65
4,HIV diagnoses,2014,United States,NaN,265732678,39963,15,7580,5.6,32383,...,38.8,42,9.5,10423,6.1,9853,2222,26428,1395,66


In [25]:
diag_master_data.to_csv('Output/master_diag.csv',index=False)

In [27]:
diag_master_data.columns

Index(['Indicator', 'Year', 'Full Name', 'FIPS', 'Population',
       'All Diag Cases', 'All Diag Rates', 'F Diag Cases', 'F Diag Rates',
       'M Diag Cases', 'M Diag Rates', '13-24 Diag Cases', '13-24 Diag Rates',
       '25-34 Diag Cases', '25-34 Diag Rates', '35-44 Diag Cases',
       '35-44 Diag Rates', '45-54 Diag Cases', '45-54 Diag Rates',
       '55+ Diag Cases', '55+ Diag Rates', 'AI_AN Diag Cases',
       'AI_AN Diag Rates', 'Asian Diag Cases', 'Asian Diag Rates',
       'Black Diag Cases', 'Black Diag Rates', 'Latino Diag Cases',
       'Latino Diag Rates', 'Multi Diag Cases', 'Multi Diag Rates',
       'NH_OPI Diag Cases', 'NH_OPI Diag Rates', 'White Diag Cases',
       'White Diag Rates', 'M2F Diag Cases', 'ID Diag Cases', 'M2M Diag Cases',
       'M2MID Diag Cases', 'O Diag Cases'],
      dtype='object')

<h3>Load dataframe to database on AWS</h3>
Went into MySQL Workbench to change data types for columns to increase efficiency.

In [ ]:
# Dataframe to SQL (is there a way to say append if not already present)
diag_master_data.to_sql(name='diag_data', if_exists='replace', con=conn, chunksize=500, index=False)

In [ ]:
remote_diag_data = pd.read_sql("SELECT * FROM diag_data", conn)
print(len(remote_diag_data))
remote_diag_data.head()